In [1]:
from dataclasses import dataclass

@dataclass
class Hailstone:
    px: int
    py: int
    pz: int
    vx: int
    vy: int
    vz: int
    slope: float
    intercept: float

In [2]:
import re

hailstones = []
with open("Day24.txt") as file:
    for line in file:
        px, py, pz, vx, vy, vz = map(int, re.findall(r"-?\d+", line))
        hailstone = Hailstone(px, py, pz, vx, vy, vz, vy / vx, py - (vy / vx) * px)
        hailstones.append(hailstone)

In [3]:
from itertools import combinations

# Intersection formula decomposition:
# 1) ax + b = cx + d
# 2) ax - cx = d - b
# 3) x(a - c) = d - b
# 4) x = (d - b) / (a - c)
count = 0
tr1, tr2 = (200_000_000_000_000, 400_000_000_000_000)
for h1, h2 in combinations(hailstones, 2):
    if h1.slope == h2.slope:
        continue  # lines are parallel
    ix = (h2.intercept - h1.intercept) / (h1.slope - h2.slope)
    iy = h1.slope * ix + h1.intercept
    t1, t2 = (ix - h1.px) / h1.vx, (ix - h2.px) / h2.vx
    if t1 < 0 or t2 < 0:
        continue  # behind starting points
    count += tr1 <= ix <= tr2 and tr1 <= iy <= tr2
count

19976

In [4]:
import z3

solver = z3.Solver()
# variables should be ints but for unknown reason z3 compute waaaaaaaay faster with reals
px_, py_, pz_, vx_, vy_, vz_ = z3.Reals("px_ py_ pz_ vx_ vy_ vz_")
for i, h in enumerate(hailstones[:3]):  # we only need to solve the equations over 3 hailstones
    t_ = z3.Real(f"t{i}")
    solver.add(t_ > 0)
    solver.add(px_ + t_ * vx_ == h.px + t_ * h.vx)
    solver.add(py_ + t_ * vy_ == h.py + t_ * h.vy)
    solver.add(pz_ + t_ * vz_ == h.pz + t_ * h.vz)
solver.check()
model = solver.model()
sum(model[var].as_long() for var in (px_, py_, pz_))

849377770236905